# Phase 1: Local Training Orchestration (Google Colab Compatible)

This notebook orchestrates all training activities for **local execution** with Google Colab GPU compute support.

## Overview

- **Step 1**: Load Centralized Configs
- **Step 2**: Verify Local Dataset (from data config)
- **Step 3**: Setup Local Environment
- **Step 4**: The Dry Run
- **Step 5**: The Sweep (HPO) - Local with Optuna
- **Step 6**: Best Configuration Selection (Automated)
- **Step 7**: Final Training (Post-HPO, Single Run)
- **Step 8**: Model Conversion & Optimization

## Important

- This notebook **executes training locally** (not on Azure ML)
- All computation happens on the local machine or Google Colab GPU
- The notebook must be **re-runnable end-to-end**
- Uses the dataset path specified in the data config (from `config/data/*.yaml`), typically pointing to a local folder included in the repository (no external downloads needed)


## Create a local Conda environment


### 1. Open a terminal in the project root

In PowerShell:

```powershell
cd "C:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml"
```

### 2. Create the Conda environment from the project’s `conda.yaml`

```powershell
conda env create -f config\environment\conda.yaml
```

- This will create an environment named `resume-ner-training` (from the `name:` field in the YAML).
- It installs Python 3.10, PyTorch, transformers, Azure ML SDK, etc.

If Conda says the env already exists, use:

```powershell
conda env update -f config\environment\conda.yaml
```

### 3. Activate the environment

```powershell
conda activate resume-ner-training
```

## Step P1-3.1: Load Centralized Configs

Load and validate all configuration files. Configs are immutable and will be logged with each job for reproducibility.


In [11]:
# # Install required packages for local execution
# %pip install kagglehub optuna mlflow --quiet


In [12]:
import os
import sys
from pathlib import Path

IN_COLAB = "COLAB_GPU" in os.environ or "COLAB_TPU" in os.environ

# Assume this notebook lives in `notebooks/` under the project root
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
SRC_DIR = ROOT_DIR / "src"
CONFIG_DIR = ROOT_DIR / "config"

sys.path.append(str(ROOT_DIR))
sys.path.append(str(SRC_DIR))

print("Notebook directory:", NOTEBOOK_DIR)
print("Project root:", ROOT_DIR)
print("Source directory:", SRC_DIR)
print("Config directory:", CONFIG_DIR)
print("In Colab:", IN_COLAB)

Notebook directory: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\notebooks
Project root: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml
Source directory: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\src
Config directory: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\config
In Colab: False


In [13]:
from pathlib import Path
from typing import Any, Dict

from orchestration import EXPERIMENT_NAME
from orchestration.config_loader import (
    ExperimentConfig,
    compute_config_hashes,
    create_config_metadata,
    load_all_configs,
    load_experiment_config,
    snapshot_configs,
    validate_config_immutability,
)

# P1-3.1: Load Centralized Configs (local-only)
# Mirrors the Azure orchestration notebook, but does not create an Azure ML client.

if not CONFIG_DIR.exists():
    raise FileNotFoundError(f"Config directory not found: {CONFIG_DIR}")

experiment_config: ExperimentConfig = load_experiment_config(CONFIG_DIR, EXPERIMENT_NAME)
configs: Dict[str, Any] = load_all_configs(experiment_config)
config_hashes = compute_config_hashes(configs)
config_metadata = create_config_metadata(configs, config_hashes)

# Immutable snapshots for runtime mutation checks
original_configs = snapshot_configs(configs)
validate_config_immutability(configs, original_configs)

print(f"Loaded experiment: {experiment_config.name}")
print("Loaded config domains:", sorted(configs.keys()))
print("Config hashes:", config_hashes)
print("Config metadata:", config_metadata)

# Get dataset path from data config (centralized configuration)
# The local_path in the data config is relative to the config directory
data_config = configs["data"]
local_path_str = data_config.get("local_path", "../dataset")
DATASET_LOCAL_PATH = (CONFIG_DIR / local_path_str).resolve()

print(f"Dataset path (from data config): {DATASET_LOCAL_PATH}")

Loaded experiment: resume_ner_baseline
Loaded config domains: ['data', 'env', 'hpo', 'model', 'train']
Config hashes: {'data': '99115e7d01de0510', 'model': '5f90a66353401b44', 'train': 'f0c2caf728759868', 'hpo': 'b28114c649d43a67', 'env': '3e54b931c7640cf2'}
Config metadata: {'data_config_hash': '99115e7d01de0510', 'model_config_hash': '5f90a66353401b44', 'train_config_hash': 'f0c2caf728759868', 'hpo_config_hash': 'b28114c649d43a67', 'env_config_hash': '3e54b931c7640cf2', 'data_version': 'v2.2', 'model_backbone': 'distilbert-base-uncased'}
Dataset path (from data config): C:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\dataset_tiny


## Step P1-3.2: Verify Local Dataset

Verify that the dataset directory (specified by `local_path` in the data config) exists and contains the required files. The dataset path is loaded from the centralized data configuration in Step P1-3.1.


In [4]:
# P1-3.2: Verify Local Dataset
# The dataset path comes from the data config's local_path field (loaded in Step P1-3.1).
# This ensures the dataset location is controlled by centralized configuration.
# Note: train.json is required, but validation.json is optional (matches training script behavior).

REQUIRED_FILE = "train.json"
OPTIONAL_FILE = "validation.json"

if not DATASET_LOCAL_PATH.exists():
    raise FileNotFoundError(
        f"Dataset directory not found: {DATASET_LOCAL_PATH}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create the dataset, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check required file
train_file = DATASET_LOCAL_PATH / REQUIRED_FILE
if not train_file.exists():
    raise FileNotFoundError(
        f"Required dataset file not found: {train_file}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create it, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check optional file
val_file = DATASET_LOCAL_PATH / OPTIONAL_FILE
has_validation = val_file.exists()

print(f"✓ Dataset directory found: {DATASET_LOCAL_PATH}")
print(f"  (from data config: {data_config.get('name', 'unknown')} v{data_config.get('version', 'unknown')})")

train_size = train_file.stat().st_size
print(f"  ✓ {REQUIRED_FILE} ({train_size:,} bytes)")

if has_validation:
    val_size = val_file.stat().st_size
    print(f"  ✓ {OPTIONAL_FILE} ({val_size:,} bytes)")
else:
    print(f"  ⚠ {OPTIONAL_FILE} not found (optional - training will proceed without validation set)")


✓ Dataset directory found: C:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\dataset_tiny
  (from data config: resume-ner-data-tiny-short vv2.2)
  ✓ train.json (13,499 bytes)
  ✓ validation.json (2,665 bytes)


## Step P1-3.3: Setup Local Environment

Verify GPU availability, set up MLflow tracking (local file store), and check that key dependencies are installed. This step ensures the local environment is ready for training.


In [5]:
import sys
import torch

DEFAULT_DEVICE = "cuda"

env_config = configs["env"]
device_type = env_config.get("compute", {}).get("device", DEFAULT_DEVICE)

if device_type == "cuda" and not torch.cuda.is_available():
    raise RuntimeError(
        "CUDA device requested but not available. "
        "Set device to 'cpu' in env config or ensure CUDA is properly installed."
    )


In [6]:
import mlflow

MLFLOW_DIR = "mlruns"
mlflow_tracking_path = ROOT_DIR / MLFLOW_DIR
mlflow_tracking_path.mkdir(exist_ok=True)

# Convert Windows path to file:// URI format for MLflow
mlflow_tracking_uri = mlflow_tracking_path.as_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)


In [7]:
try:
    import transformers
    import optuna
except ImportError as e:
    raise ImportError(f"Required package not installed: {e}")

REQUIRED_PACKAGES = {
    "torch": torch,
    "transformers": transformers,
    "mlflow": mlflow,
    "optuna": optuna,
}

for name, module in REQUIRED_PACKAGES.items():
    if not hasattr(module, "__version__"):
        raise ImportError(f"Required package '{name}' is not properly installed")


c:\Users\HOANG PHI LONG DANG\Miniconda3\envs\resume-ner-training\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step P1-3.4: The Dry Run

Run a minimal HPO sweep to validate the training pipeline works correctly before launching the full HPO sweep. Uses the smoke HPO configuration with reduced trials.


In [8]:
from pathlib import Path
from orchestration import STAGE_SMOKE
from orchestration.jobs.local_sweeps import run_local_hpo_sweep

TRAINING_SCRIPT_PATH = SRC_DIR / "train.py"
DRY_RUN_OUTPUT_DIR = ROOT_DIR / "outputs" / "dry_run"

if not TRAINING_SCRIPT_PATH.exists():
    raise FileNotFoundError(f"Training script not found: {TRAINING_SCRIPT_PATH}")


In [9]:
hpo_config = configs["hpo"]
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]

dry_run_studies = {}

for backbone in backbone_values:
    mlflow_experiment_name = f"{experiment_config.name}-{STAGE_SMOKE}-{backbone}"
    backbone_output_dir = DRY_RUN_OUTPUT_DIR / backbone
    
    study = run_local_hpo_sweep(
        dataset_path=str(DATASET_LOCAL_PATH),
        config_dir=CONFIG_DIR,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_dir=backbone_output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
    )
    
    dry_run_studies[backbone] = study


[I 2025-12-19 00:15:21,033] A new study created in memory with name: hpo_deberta
  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\HOANG PHI LONG DANG\Miniconda3\envs\resume-ner-training\lib\site-packages\mlflow\tracking\_tracking_service\utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/19 00:15:21 INFO mlflow.tracking.fluent: Experiment with name 'resume_ner_baseline-smoke-deberta' does not exist. Creating a new experiment.
Best trial: 0. Best value: 0.068687:  50%|█████     | 1/2 [03:23<03:23, 203.35s/it, 203.35/1200 seconds]

[I 2025-12-19 00:18:44,387] Trial 0 finished with value: 0.06868697262811278 and parameters: {'backbone': 'deberta', 'learning_rate': 1.2780420377992051e-05, 'batch_size': 4, 'dropout': 0.18252475295510473, 'weight_decay': 0.051208753785545}. Best is trial 0 with value: 0.06868697262811278.


Best trial: 1. Best value: 0.153773: 100%|██████████| 2/2 [03:43<00:00, 111.83s/it, 223.67/1200 seconds]

[I 2025-12-19 00:19:04,702] Trial 1 finished with value: 0.15377348779855068 and parameters: {'backbone': 'deberta', 'learning_rate': 1.9319872207119493e-05, 'batch_size': 4, 'dropout': 0.10385127864214172, 'weight_decay': 0.001801170452800572}. Best is trial 1 with value: 0.15377348779855068.


In [ ]:
for backbone, study in dry_run_studies.items():
    if study.trials:
        best_trial = study.best_trial
        print(f"{backbone}: {len(study.trials)} trials completed")
        print(
            f"  Best {hpo_config['objective']['metric']}: {best_trial.value:.4f}")
        print(f"  Best params: {best_trial.params}")
    else:
        print(f"{backbone}: No trials completed")

## Step P1-3.5: The Sweep (HPO) - Local with Optuna

Run the full hyperparameter optimization sweep using Optuna to systematically search for the best model configuration. Uses the production HPO configuration with more trials than the dry run.


In [14]:
from pathlib import Path
from orchestration import STAGE_HPO
from shared.yaml_utils import load_yaml
from orchestration.jobs.local_sweeps import run_local_hpo_sweep

HPO_OUTPUT_DIR = ROOT_DIR / "outputs" / "hpo"

hpo_stage_config = experiment_config.stages.get(STAGE_HPO, {})
hpo_config_override = hpo_stage_config.get("hpo_config")

if hpo_config_override:
    hpo_config_path = CONFIG_DIR / hpo_config_override
else:
    hpo_config_path = experiment_config.hpo_config

if not hpo_config_path.exists():
    raise FileNotFoundError(f"HPO config not found: {hpo_config_path}")


In [15]:
hpo_config = load_yaml(hpo_config_path)
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]


In [16]:
hpo_config

{'search_space': {'backbone': {'type': 'choice', 'values': ['deberta']},
  'learning_rate': {'type': 'loguniform', 'min': '1e-5', 'max': '5e-5'},
  'batch_size': {'type': 'choice', 'values': [4]},
  'dropout': {'type': 'uniform', 'min': 0.1, 'max': 0.3},
  'weight_decay': {'type': 'loguniform', 'min': 0.001, 'max': 0.1}},
 'sampling': {'algorithm': 'random', 'max_trials': 2, 'timeout_minutes': 20},
 'early_termination': {'policy': 'bandit',
  'evaluation_interval': 1,
  'slack_factor': 0.2,
  'delay_evaluation': 2},
 'objective': {'metric': 'macro-f1', 'goal': 'maximize'}}

In [17]:
hpo_studies = {}

for backbone in backbone_values:
    mlflow_experiment_name = f"{experiment_config.name}-{STAGE_HPO}-{backbone}"
    backbone_output_dir = HPO_OUTPUT_DIR / backbone
    
    study = run_local_hpo_sweep(
        dataset_path=str(DATASET_LOCAL_PATH),
        config_dir=CONFIG_DIR,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_dir=backbone_output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
    )
    
    hpo_studies[backbone] = study


[I 2025-12-19 00:42:54,169] A new study created in memory with name: hpo_deberta
  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\HOANG PHI LONG DANG\Miniconda3\envs\resume-ner-training\lib\site-packages\mlflow\tracking\_tracking_service\utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/19 00:42:54 INFO mlflow.tracking.fluent: Experiment with name 'resume_ner_baseline-hpo-deberta' does not exist. Creating a new experiment.
Best trial: 0. Best value: 0.0804882:  50%|█████     | 1/2 [00:22<00:22, 22.21s/it, 22.21/1200 seconds]

[I 2025-12-19 00:43:16,376] Trial 0 finished with value: 0.0804881924580967 and parameters: {'backbone': 'deberta', 'learning_rate': 1.2288688273017396e-05, 'batch_size': 4, 'dropout': 0.2838947702094308, 'weight_decay': 0.020483659291694205}. Best is trial 0 with value: 0.0804881924580967.


Best trial: 1. Best value: 0.216243: 100%|██████████| 2/2 [00:43<00:00, 21.70s/it, 43.40/1200 seconds] 

[I 2025-12-19 00:43:37,574] Trial 1 finished with value: 0.21624301624301623 and parameters: {'backbone': 'deberta', 'learning_rate': 2.1861220321511983e-05, 'batch_size': 4, 'dropout': 0.2770894954670945, 'weight_decay': 0.0327476003023506}. Best is trial 1 with value: 0.21624301624301623.


In [18]:
for backbone, study in hpo_studies.items():
    if study.trials:
        best_trial = study.best_trial
        print(f"{backbone}: {len(study.trials)} trials completed")
        print(f"  Best {hpo_config['objective']['metric']}: {best_trial.value:.4f}")
        print(f"  Best params: {best_trial.params}")
    else:
        print(f"{backbone}: No trials completed")


deberta: 2 trials completed
  Best macro-f1: 0.2162
  Best params: {'backbone': 'deberta', 'learning_rate': 2.1861220321511983e-05, 'batch_size': 4, 'dropout': 0.2770894954670945, 'weight_decay': 0.0327476003023506}


## Step P1-3.6: Best Configuration Selection (Automated)

Programmatically select the best configuration from all HPO sweep runs across all backbone models. The best configuration is determined by the objective metric specified in the HPO config.


In [20]:
from pathlib import Path
from shared.json_cache import save_json
from orchestration.jobs.local_selection import select_best_configuration_across_studies

BEST_CONFIG_CACHE_FILE = ROOT_DIR / "notebooks" / "best_configuration_cache.json"


In [21]:
dataset_version = data_config.get("version", "unknown")

best_configuration = select_best_configuration_across_studies(
    studies=hpo_studies,
    hpo_config=hpo_config,
    dataset_version=dataset_version,
)


In [22]:
save_json(BEST_CONFIG_CACHE_FILE, best_configuration)

print(f"Best configuration selected:")
print(f"  Backbone: {best_configuration.get('backbone')}")
print(f"  Trial: {best_configuration.get('trial_name')}")
print(f"  Best {hpo_config['objective']['metric']}: {best_configuration.get('selection_criteria', {}).get('best_value'):.4f}")
print(f"  Hyperparameters: {best_configuration.get('hyperparameters')}")
print(f"\nSaved to: {BEST_CONFIG_CACHE_FILE}")


Best configuration selected:
  Backbone: deberta
  Trial: trial_1
  Best macro-f1: 0.2162
  Hyperparameters: {'learning_rate': 2.1861220321511983e-05, 'batch_size': 4, 'dropout': 0.2770894954670945, 'weight_decay': 0.0327476003023506}

Saved to: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\notebooks\best_configuration_cache.json


## Step P1-3.7: Final Training (Post-HPO, Single Run)

Train the final production model using the best configuration from HPO with stable, controlled conditions. This uses the full training epochs (no early stopping) and the best hyperparameters found during HPO.


In [26]:
from pathlib import Path
import os
import sys
import subprocess
import mlflow
from shared.json_cache import load_json, save_json
from orchestration import STAGE_TRAINING
from orchestration.jobs.training import build_final_training_config

DEFAULT_RANDOM_SEED = 42
BEST_CONFIG_CACHE_FILE = ROOT_DIR / "notebooks" / "best_configuration_cache.json"
FINAL_TRAINING_OUTPUT_DIR = ROOT_DIR / "outputs" / "final_training"


In [27]:
best_configuration = load_json(BEST_CONFIG_CACHE_FILE, default=None)

if best_configuration is None:
    raise FileNotFoundError(
        f"Best configuration cache not found: {BEST_CONFIG_CACHE_FILE}\n"
        f"Please run Step P1-3.6: Best Configuration Selection first."
    )

final_training_config = build_final_training_config(
    best_config=best_configuration,
    train_config=configs["train"],
    random_seed=DEFAULT_RANDOM_SEED,
)


In [28]:
mlflow_experiment_name = f"{experiment_config.name}-{STAGE_TRAINING}-{final_training_config['backbone']}"
final_output_dir = FINAL_TRAINING_OUTPUT_DIR / final_training_config['backbone']
final_output_dir.mkdir(parents=True, exist_ok=True)

mlflow.set_experiment(mlflow_experiment_name)


2025/12/19 01:09:01 INFO mlflow.tracking.fluent: Experiment with name 'resume_ner_baseline-training-deberta' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/HOANG%20PHI%20LONG%20DANG/repos/resume-ner-azureml/mlruns/248982361299698356', creation_time=1766102941733, experiment_id='248982361299698356', last_update_time=1766102941733, lifecycle_stage='active', name='resume_ner_baseline-training-deberta', tags={}>

In [29]:
training_script_path = SRC_DIR / "train.py"
training_args = [
    sys.executable,
    str(training_script_path),
    "--data-asset",
    str(DATASET_LOCAL_PATH),
    "--config-dir",
    str(CONFIG_DIR),
    "--backbone",
    final_training_config["backbone"],
    "--learning-rate",
    str(final_training_config["learning_rate"]),
    "--batch-size",
    str(final_training_config["batch_size"]),
    "--dropout",
    str(final_training_config["dropout"]),
    "--weight-decay",
    str(final_training_config["weight_decay"]),
    "--epochs",
    str(final_training_config["epochs"]),
    "--random-seed",
    str(final_training_config["random_seed"]),
    "--early-stopping-enabled",
    str(final_training_config["early_stopping_enabled"]).lower(),
    "--use-combined-data",
    str(final_training_config["use_combined_data"]).lower(),
]


In [30]:
training_env = os.environ.copy()
training_env["AZURE_ML_OUTPUT_checkpoint"] = str(final_output_dir)

mlflow_tracking_uri = mlflow.get_tracking_uri()
if mlflow_tracking_uri:
    training_env["MLFLOW_TRACKING_URI"] = mlflow_tracking_uri
training_env["MLFLOW_EXPERIMENT_NAME"] = mlflow_experiment_name


In [31]:
result = subprocess.run(
    training_args,
    cwd=ROOT_DIR,
    env=training_env,
    capture_output=False,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Final training failed with return code {result.returncode}")


In [32]:
import json

METRICS_FILENAME = "metrics.json"
FINAL_TRAINING_CACHE_FILE = ROOT_DIR / "notebooks" / "final_training_cache.json"

metrics_file = final_output_dir / METRICS_FILENAME
if metrics_file.exists():
    with open(metrics_file, "r") as f:
        metrics = json.load(f)
    print(f"✓ Training completed. Checkpoint: {final_output_dir}")
else:
    raise FileNotFoundError(f"Training metrics not found: {metrics_file}")

save_json(FINAL_TRAINING_CACHE_FILE, {
    "output_dir": str(final_output_dir),
    "backbone": final_training_config["backbone"],
    "config": final_training_config,
})


✓ Training completed. Checkpoint: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\outputs\final_training\deberta


## Step P1-4: Model Conversion & Optimization

Convert the final training checkpoint to an optimized ONNX model (int8 quantized) for production inference.

**Platform Adapter Note**: The conversion script (`src/convert_to_onnx.py`) uses the platform adapter to automatically handle output paths and logging appropriately for local execution.


In [33]:
from pathlib import Path
import os
import sys
import subprocess
import mlflow
from shared.json_cache import load_json

CONVERSION_SCRIPT_PATH = SRC_DIR / "convert_to_onnx.py"
FINAL_TRAINING_CACHE_FILE = ROOT_DIR / "notebooks" / "final_training_cache.json"
CONVERSION_OUTPUT_DIR = ROOT_DIR / "outputs" / "conversion"


In [34]:
training_cache = load_json(FINAL_TRAINING_CACHE_FILE, default=None)

if training_cache is None:
    raise FileNotFoundError(
        f"Final training cache not found: {FINAL_TRAINING_CACHE_FILE}\n"
        f"Please run Step P1-3.7: Final Training first."
    )

checkpoint_dir = Path(training_cache["output_dir"]) / "checkpoint"
if not checkpoint_dir.exists():
    raise FileNotFoundError(f"Checkpoint directory not found: {checkpoint_dir}")

backbone = training_cache["backbone"]
conversion_output_dir = CONVERSION_OUTPUT_DIR / backbone
conversion_output_dir.mkdir(parents=True, exist_ok=True)


In [35]:
conversion_args = [
    sys.executable,
    str(CONVERSION_SCRIPT_PATH),
    "--checkpoint-path",
    str(checkpoint_dir),
    "--config-dir",
    str(CONFIG_DIR),
    "--backbone",
    backbone,
    "--output-dir",
    str(conversion_output_dir),
    "--quantize-int8",
    "--run-smoke-test",
]


In [36]:
conversion_env = os.environ.copy()
conversion_env["AZURE_ML_OUTPUT_onnx_model"] = str(conversion_output_dir)

mlflow_tracking_uri = mlflow.get_tracking_uri()
if mlflow_tracking_uri:
    conversion_env["MLFLOW_TRACKING_URI"] = mlflow_tracking_uri


In [37]:
result = subprocess.run(
    conversion_args,
    cwd=ROOT_DIR,
    env=conversion_env,
    capture_output=False,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Model conversion failed with return code {result.returncode}")


In [38]:
from shared.json_cache import save_json

ONNX_MODEL_FILENAME = "model_int8.onnx"
FALLBACK_ONNX_MODEL_FILENAME = "model.onnx"
CONVERSION_CACHE_FILE = ROOT_DIR / "notebooks" / "conversion_cache.json"

onnx_model_path = conversion_output_dir / ONNX_MODEL_FILENAME
if not onnx_model_path.exists():
    onnx_model_path = conversion_output_dir / FALLBACK_ONNX_MODEL_FILENAME

if not onnx_model_path.exists():
    raise FileNotFoundError(f"ONNX model not found in {conversion_output_dir}")

print(f"✓ Conversion completed. ONNX model: {onnx_model_path}")

save_json(CONVERSION_CACHE_FILE, {
    "onnx_model_path": str(onnx_model_path),
    "backbone": backbone,
    "checkpoint_dir": str(checkpoint_dir),
})


✓ Conversion completed. ONNX model: c:\Users\HOANG PHI LONG DANG\repos\resume-ner-azureml\outputs\conversion\deberta\model_int8.onnx
